### cho tập dữ liệu mẫu như dưới. dùng RDD giải các bài toán sau:
1. Tính tổng số tiền mua hàng của tất cả các khách hàng.
2. Tính tổng số tiền mua hàng của các khách hàng trong từng thành phố.
3. Tìm khách hàng có số tiền mua hàng cao nhất và thấp nhất.
4. Tính tuổi trung bình của các khách hàng.
5. Tính tuổi trung bình của các khách hàng theo giới tính.
6. Tìm các khách hàng có tuổi lớn hơn 30 và số tiền mua hàng lớn hơn 400.
7. Sắp xếp danh sách khách hàng theo số tiền mua hàng giảm dần.
8. Đếm số lượng khách hàng trong mỗi thành phố.
9. Đếm số lượng khách hàng theo từng giới tính.
10. Tìm các khách hàng có số tiền mua hàng lớn hơn 500.
11. Tính số tiền mua hàng trung bình của các khách hàng trong mỗi thành phố.
12. Tìm khách hàng trẻ nhất và già nhất trong tập dữ liệu.
13. Tính tổng số tiền mua hàng của các khách hàng có tuổi từ 25 đến 35.
14. Tính tổng số tiền mua hàng của các khách hàng theo từng giới tính.
15. Đếm số lượng khách hàng có số tiền mua hàng lớn hơn 300 trong mỗi thành phố.
16. Tìm các khách hàng có tên bắt đầu bằng chữ "J".
17. Tính số tiền mua hàng trung bình của các khách hàng có tuổi lớn hơn 40.
18. Tạo một danh sách các thành phố khác nhau có trong tập dữ liệu.
19. Tính tổng số tiền mua hàng của các khách hàng có tên chứa chữ "a".
20. Tính tổng số tiền mua hàng của các khách hàng trong từng thành phố và sắp xếp các thành phố theo tổng số tiền mua hàng giảm dần.

In [31]:
column = ["id" , "name", "age", "gender", "purchase_amount", "location"]
data = [
    "1,John Doe,28,Male,300,New York",
    "2,Jane Smith,34,Female,450,Los Angeles",
    "3,Jim Brown,45,Male,700,Chicago",
    "4,Lucy Black,23,Female,120,New York",
    "5,Jack White,31,Male,350,Los Angeles",
    "6,Emily Green,27,Female,200,Chicago",
    "7,James Blue,36,Male,400,New York",
    "8,Olivia Yellow,22,Female,300,Los Angeles",
    "9,Robert Red,50,Male,900,Chicago",
    "10,Mary Purple,29,Female,600,New York"
]


### tao sc

In [1]:
from pyspark.sql import SparkSession
ss = SparkSession.builder \
    .appName("thuc hanh RDD") \
    .master("local[*]") \
    .getOrCreate()

In [2]:
sc = ss.sparkContext

### Load data

In [32]:
rdd = sc.parallelize(data)

In [33]:
rdd.take(5)

['1,John Doe,28,Male,300,New York',
 '2,Jane Smith,34,Female,450,Los Angeles',
 '3,Jim Brown,45,Male,700,Chicago',
 '4,Lucy Black,23,Female,120,New York',
 '5,Jack White,31,Male,350,Los Angeles']

### 1. Tính tổng số tiền mua hàng của tất cả các khách hàng.


In [8]:
so_tien_mua_hang_rdd = rdd.map(lambda x: x.split(",")[4])
so_tien_mua_hang_rdd.take(5)

['300', '450', '700', '120', '350']

In [11]:
tong_so_tien_mua_hang_rdd = so_tien_mua_hang_rdd.reduce(lambda x, y: int(x) + int(y))
print(tong_so_tien_mua_hang_rdd)

4320


### 2. Tính tổng số tiền mua hàng của các khách hàng trong từng thành phố.


In [14]:
so_tien_mua_hang_theo_thanh_pho_rdd = rdd.map(lambda x: (x.split(",")[5], x.split(",")[4]))
so_tien_mua_hang_theo_thanh_pho_rdd.take(5)

[('New York', '300'),
 ('Los Angeles', '450'),
 ('Chicago', '700'),
 ('New York', '120'),
 ('Los Angeles', '350')]

In [16]:
# reduceByKey giảm key trên toàn bộ RDD, nên gây ra nhiều sự trao đổi dữ liệu giữa các phân vùng rồi mới tính toán
tong_tien_mua_hang_theo_thanh_pho_rdd = so_tien_mua_hang_theo_thanh_pho_rdd.reduceByKey(lambda x, y: int(x) + int(y))
tong_tien_mua_hang_theo_thanh_pho_rdd.take(10)

[('Los Angeles', 1100), ('Chicago', 1800), ('New York', 1420)]

In [19]:
# reduceByKeyLocally giảm key trên từng phân vùng trước, xong tính toán, xong mới chuyển dữ liệu giữa các phân vùng nên lượng dữ liệu
# trao đổi giữa các phân vùng ít hơn(do đã được giảm) -> nhanh hơn 5s so với reduceByKey trong ví dụ này
tong_tien_mua_hang_theo_thanh_pho_rdd_2 = so_tien_mua_hang_theo_thanh_pho_rdd.reduceByKeyLocally(lambda x, y: int(x) + int(y))
print(tong_tien_mua_hang_theo_thanh_pho_rdd_2)

{'New York': 1420, 'Los Angeles': 1100, 'Chicago': 1800}


### 3. Tìm khách hàng có số tiền mua hàng cao nhất và thấp nhất.


In [20]:
khach_hang_vs_tien_mua_hang_rdd = rdd.map(lambda x: (x.split(",")[0], x.split(",")[4]))
khach_hang_vs_tien_mua_hang_rdd.take(5)

[('1', '300'), ('2', '450'), ('3', '700'), ('4', '120'), ('5', '350')]

In [22]:
sort_so_tien_mua_hang_rdd = khach_hang_vs_tien_mua_hang_rdd.sortBy(lambda x: x[1], False)
sort_so_tien_mua_hang_rdd.take(20)

[('9', '900'),
 ('3', '700'),
 ('10', '600'),
 ('2', '450'),
 ('7', '400'),
 ('5', '350'),
 ('1', '300'),
 ('8', '300'),
 ('6', '200'),
 ('4', '120')]

In [36]:
khach_mua_so_tien_mua_cao_nhat = rdd.filter(lambda x: x.split(",")[0] == "9")
khach_mua_so_tien_mua_thap_nhat = rdd.filter(lambda x: x.split(",")[0] == "4")
print("khach hang co so tien mua cao nhat: ", khach_mua_so_tien_mua_cao_nhat.take(1))
print("khach hang co so tien mua thap nhat: ", khach_mua_so_tien_mua_thap_nhat.take(1))

khach hang co so tien mua cao nhat:  ['9,Robert Red,50,Male,900,Chicago']
khach hang co so tien mua thap nhat:  ['4,Lucy Black,23,Female,120,New York']


### 4. Tính tuổi trung bình của các khách hàng.


In [37]:
tuoi_khach_hang_rdd = rdd.map(lambda x: x.split(",")[2])
tuoi_khach_hang_rdd.take(5)

['28', '34', '45', '23', '31']

In [39]:
so_luong_khach = rdd.count()

In [43]:
tong_tuoi_khach = tuoi_khach_hang_rdd.reduce(lambda x, y: int(x) + int(y))
print(tong_tuoi_khach)

325


In [44]:
print("trung binh tuoi cua khach hang la: ", round(tong_tuoi_khach/so_luong_khach, 2))

trung binh tuoi cua khach hang la:  32.5


### 5. Tính tuổi trung bình của các khách hàng theo giới tính.


In [55]:
gioi_tinh_vs_tuoi_rdd = rdd.map(lambda x: (x.split(",")[3], x.split(",")[2]))
gioi_tinh_vs_tuoi_rdd.take(5)

[('Male', '28'),
 ('Female', '34'),
 ('Male', '45'),
 ('Female', '23'),
 ('Male', '31')]

In [56]:
# split 1 lan, toi uu thoi gian tinh toan
def extract_sex_age(line):
    temp = line.split(",")
    return (temp[3], temp[2])
gioi_tinh_vs_tuoi_rdd_2 = rdd.map(extract_sex_age)
gioi_tinh_vs_tuoi_rdd_2.take(5)

[('Male', '28'),
 ('Female', '34'),
 ('Male', '45'),
 ('Female', '23'),
 ('Male', '31')]

In [58]:
tong_tuoi_theo_gioi_tinh_rdd = gioi_tinh_vs_tuoi_rdd.reduceByKeyLocally(lambda x, y: int(x) + int(y))
print(tong_tuoi_theo_gioi_tinh_rdd)

{'Male': 190, 'Female': 135}


In [61]:
count_male = gioi_tinh_vs_tuoi_rdd.filter(lambda x: x[0] == "Male").count()
print(count_male)

5


In [62]:
print("trung binh nam gioi: ", round(int(tong_tuoi_theo_gioi_tinh_rdd["Male"])/count_male, 2), "tuoi")
print("trung binh nu gioi: ", round(int(tong_tuoi_theo_gioi_tinh_rdd["Female"])/(so_luong_khach - count_male), 2), "tuoi")

trung binh nam gioi:  38.0 tuoi
trung binh nu gioi:  27.0 tuoi


### 6. Tìm các khách hàng có tuổi lớn hơn 30 và số tiền mua hàng lớn hơn 400.


In [74]:
def filter_age_purchase_amount(line):
    line_split = line.split(",")
    if int(line_split[2]) > 30 and int(line_split[4]) > 400:
        return line
    else:
        pass

In [75]:
age_purchase_amount_rdd = rdd.map(filter_age_purchase_amount)
age_purchase_amount_rdd.take(5)

[None,
 '2,Jane Smith,34,Female,450,Los Angeles',
 '3,Jim Brown,45,Male,700,Chicago',
 None,
 None]

In [81]:
age_purchase_amount_rdd.filter(lambda x: x != None).take(10)

['2,Jane Smith,34,Female,450,Los Angeles',
 '3,Jim Brown,45,Male,700,Chicago',
 '9,Robert Red,50,Male,900,Chicago']

### 8. Đếm số lượng khách hàng trong mỗi thành phố.


In [34]:
# sẽ có trường hợp khách hàng không có thông tin thành phố, thành phố bị null
loai_bo_null_tp_rdd = rdd.filter(lambda x: x.split(",")[-1] != "Null")
loai_bo_null_tp_rdd.take(10)

['1,John Doe,28,Male,300,New York',
 '2,Jane Smith,34,Female,450,Los Angeles',
 '3,Jim Brown,45,Male,700,Chicago',
 '4,Lucy Black,23,Female,120,New York',
 '5,Jack White,31,Male,350,Los Angeles',
 '6,Emily Green,27,Female,200,Chicago',
 '7,James Blue,36,Male,400,New York',
 '8,Olivia Yellow,22,Female,300,Los Angeles',
 '9,Robert Red,50,Male,900,Chicago',
 '10,Mary Purple,29,Female,600,New York']

In [36]:
# sẽ có trường hợp trong cùng 1 thành phố, tồn tại những hàng bị trùng id khách hàng, tao 1 tuple id va info khach
id_info_rdd = rdd.map(lambda x: (x.split(",")[0], x))
id_info_rdd.take(5)

[('1', '1,John Doe,28,Male,300,New York'),
 ('2', '2,Jane Smith,34,Female,450,Los Angeles'),
 ('3', '3,Jim Brown,45,Male,700,Chicago'),
 ('4', '4,Lucy Black,23,Female,120,New York'),
 ('5', '5,Jack White,31,Male,350,Los Angeles')]

In [44]:
distinct_id_rdd = id_info_rdd.reduceByKey(lambda x, y: y)
distinct_id_rdd = distinct_id_rdd.map(lambda x: x[1])
distinct_id_rdd.take(10)

['4,Lucy Black,23,Female,120,New York',
 '10,Mary Purple,29,Female,600,New York',
 '3,Jim Brown,45,Male,700,Chicago',
 '6,Emily Green,27,Female,200,Chicago',
 '7,James Blue,36,Male,400,New York',
 '1,John Doe,28,Male,300,New York',
 '8,Olivia Yellow,22,Female,300,Los Angeles',
 '9,Robert Red,50,Male,900,Chicago',
 '2,Jane Smith,34,Female,450,Los Angeles',
 '5,Jack White,31,Male,350,Los Angeles']

In [45]:
thanh_pho_mapped = distinct_id_rdd.map(lambda x: (x.split(",")[-1], 1))
thanh_pho_mapped.take(5)

[('New York', 1),
 ('New York', 1),
 ('Chicago', 1),
 ('Chicago', 1),
 ('New York', 1)]

In [46]:
thanh_pho_reduce_by_key = thanh_pho_mapped.reduceByKey(lambda x, y: x + y)
thanh_pho_reduce_by_key.take(5)

[('Chicago', 3), ('Los Angeles', 3), ('New York', 4)]

### 9. Đếm số lượng khách hàng theo từng giới tính.


In [47]:
gioi_tinh_mapped = rdd.map(lambda x: (x.split(",")[3], 1))
gioi_tinh_reduce_by_key = gioi_tinh_mapped.reduceByKey(lambda x, y: x + y)
gioi_tinh_reduce_by_key.take(3)

[('Male', 5), ('Female', 5)]

### 10. Tìm các khách hàng có số tiền mua hàng lớn hơn 500.


In [51]:
rdd.filter(lambda x: int(x.split(",")[4]) > 500).take(10)

['3,Jim Brown,45,Male,700,Chicago',
 '9,Robert Red,50,Male,900,Chicago',
 '10,Mary Purple,29,Female,600,New York']

### 11. Tính số tiền mua hàng trung bình của các khách hàng trong mỗi thành phố.
